In [ ]:
# Import Libraries
import pandas as pd
from faker import Faker
from faker.providers import BaseProvider
from datetime import datetime, timedelta
import random

In [ ]:
# Custom provider for product categories
class ProductProvider(BaseProvider):
    def product_name(self, category):
        products = {
            "Hijab": ["Square Hijab", "Pashmina Hijab", "Instant Hijab", "Printed Hijab"],
            "Pakaian": ["Tunik", "Rok", "Dress", "Kemeja"],
            "Mukena": ["Mukena Sutra", "Mukena Travel", "Mukena Anak", "Mukena Katun"]
        }
        return random.choice(products[category])

# Initialize Faker and add the custom provider
fake = Faker('id_ID')
fake.add_provider(ProductProvider)

# Choose categories randomly
categories = ["Hijab", "Pakaian", "Mukena"]

# Define customer types
customer_types = ["online", "offline", "reseller"]

# Generate customer data
customer_data = {
    'customer_id': [i for i in range(1, 501)],
    'customer_name': [fake.name() for _ in range(500)],
    'customer_address': [fake.address() for _ in range(500)],
    'customer_email': [fake.email() for _ in range(500)],
    'customer_phone': [fake.phone_number() for _ in range(500)],
    'customer_type': [random.choice(customer_types) for _ in range(500)],  # Randomly assign customer type
}

customers_df = pd.DataFrame(customer_data)

# Define date range for the last 2 years
start_date = datetime.now() - timedelta(days=730)
end_date = datetime.now()

# Generate order data with repeat orders and consistent product details
order_data = {
    'order_id': list(range(1, 5001)),  # Generate unique order_ids from 1 to 5000
    'customer_id': [random.choice(customer_data['customer_id']) for _ in range(5000)],  # Randomly assign customer_id
    'order_date': [fake.date_time_between(start_date=start_date, end_date=end_date) for _ in range(5000)],
    'order_qty': [fake.pyint(min_value=1, max_value=10) for _ in range(5000)],  # Ensure order_qty is reasonable
    'product_category': [random.choice(categories) for _ in range(5000)],  # Randomly assign product category
}

# Dictionary to store product details based on product name
product_details = {}

# Generate product details
product_names = [fake.product_name(category) for category in order_data['product_category']]
product_ids = [f"PROD-{random.randint(1000, 9999)}" for _ in range(5000)]
product_prices = [fake.pyint(min_value=10000, max_value=100000) for _ in range(5000)]

for name, id, price in zip(product_names, product_ids, product_prices):
    if name not in product_details:
        product_details[name] = {'product_id': id, 'product_price': price}

order_data['product_name'] = product_names
order_data['product_id'] = [product_details[name]['product_id'] for name in product_names]
order_data['product_price'] = [product_details[name]['product_price'] for name in product_names]

# Calculate total_price as the multiplication of order_qty and product_price
order_data['total_order'] = [order_data['order_qty'][i] * order_data['product_price'][i] for i in range(5000)]

orders_df = pd.DataFrame(order_data)

# Merge customer and order data
merged_df = pd.merge(orders_df, customers_df, on='customer_id')

merged_df.head(15)


,order_id,customer_id,order_date,order_qty,product_category,product_name,product_id,product_price,total_order,customer_name,customer_address,customer_email,customer_phone,customer_type
0,1,41,2024-05-28 17:21:22,10,Mukena,Mukena Anak,PROD-8344,51631,516310,Gading Zulaika,"Jalan Cikapayang No. 45\nProbolinggo, Kepulaua...",opheliasimanjuntak@example.org,+62-74-812-7387,online
1,2,115,2023-03-26 12:58:57,8,Mukena,Mukena Anak,PROD-8344,51631,413048,"Hj. Ulya Zulaika, S.Farm","Gang Erlangga No. 22\nBatam, JK 43227",yance57@example.org,+62 (562) 776 1012,reseller
2,3,177,2025-03-01 00:10:28,2,Mukena,Mukena Travel,PROD-7512,76607,153214,Anastasia Fujiati,"Gang Suniaraja No. 6\nBekasi, Aceh 84662",hamzah30@example.org,(0882) 086 8226,offline
3,4,226,2023-11-01 10:28:34,1,Pakaian,Dress,PROD-9465,14701,14701,Tira Yuniar,"Gg. Rawamangun No. 26\nCimahi, Nusa Tenggara B...",yance79@example.net,(029) 048-8066,reseller
4,5,116,2023-07-15 09:03:37,3,Mukena,Mukena Travel,PROD-7512,76607,229821,dr. Jindra Mustofa,"Gang Pelajar Pejuang No. 0\nDepok, BT 62018",fmahendra@example.com,(090) 893 9881,online
5,6,10,2024-02-10 16:05:03,2,Pakaian,Kemeja,PROD-7595,22985,45970,"Karya Habibi, M.Ak","Gg. Kapten Muslihat No. 82\nAmbon, Jawa Timur ...",ozysamosir@example.org,084 987 8996,online
6,7,92,2024-06-09 18:10:14,1,Pakaian,Kemeja,PROD-7595,22985,22985,Hendri Putra,"Gg. K.H. Wahid Hasyim No. 07\nPangkalpinang, D...",bnuraini@example.org,+62 (089) 193-7715,reseller
7,8,216,2025-01-03 01:54:50,8,Mukena,Mukena Anak,PROD-8344,51631,413048,H. Darmaji Prasetya,"Gang Cikapayang No. 36\nYogyakarta, JA 10746",surya20@example.net,0842340217,online
8,9,75,2024-01-14 18:05:20,4,Hijab,Printed Hijab,PROD-4270,38697,154788,Diana Mandala,"Gang Pacuan Kuda No. 27\nPekanbaru, Kepulauan ...",dipa34@example.org,+62 (82) 804-1022,offline
9,10,35,2023-05-10 05:45:46,8,Hijab,Printed Hijab,PROD-4270,38697,309576,Keisha Rahimah,"Gang Siliwangi No. 9\nMetro, Kalimantan Barat ...",heryanto33@example.net,+62 (034) 207 3622,online


In [56]:
# Save DataFrame to an Excel file
excel_file_path = 'sample_dataa.xlsx'
merged_df.to_excel(excel_file_path, index=False)

print(f"DataFrame successfully saved to {excel_file_path}")

DataFrame successfully saved to sample_dataa.xlsx
